In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


cpu


In [3]:
#hyperparameters
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

In [4]:
with open ('../wizard_of_oz.txt', 'r', encoding='utf-8' ) as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [5]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { ch:i for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: [int_to_string[c] for c in l]

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [6]:
n = int(0.8*len(data))
train_data = data[n:]
val_data = data[:n]

In [7]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size, ))
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    x, y = x.to(device) , y.to(device)
    return x, y

In [8]:
x , y = get_batch('train')
print('inputs : ')
#print(shape)
print(x)
print('targets : ')
print(y)
# print(y.shape)

inputs : 
tensor([[78,  1, 75, 68, 62, 56, 58, 11],
        [65, 64, 58, 57,  1, 65, 68, 67],
        [73, 61, 58,  1, 29, 66, 58, 71],
        [68, 73,  1, 68, 59, 73, 58, 67]])
targets : 
tensor([[ 1, 75, 68, 62, 56, 58, 11,  0],
        [64, 58, 57,  1, 65, 68, 67, 60],
        [61, 58,  1, 29, 66, 58, 71, 54],
        [73,  1, 68, 59, 73, 58, 67, 11]])


In [20]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [28]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)  # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T)
        for _ in range(max_new_tokens):
            logits, _ = self.forward(index)           # (B, T, C)
            logits = logits[:, -1, :]                 # take last time step -> (B, C)
            probs = F.softmax(logits, dim=-1)         # (B, C)
            index_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            index = torch.cat((index, index_next), dim=1)
        return index

# re-create model on device
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# fix int->char mapping and decode for printing
# int_to_string = { i:ch for i,ch in enumerate(chars) }
# decode = lambda l: ''.join(int_to_string[i] for i in l)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_indices = m.generate(context, max_new_tokens=500)[0].tolist()
generated_chars = decode(generated_indices)
print(generated_chars)



w6j'0h'vqNu6 Ge"(n&PTX7W;oAQ"M8f;.Lfu﻿Tz)55
ig7odDhH.Lw PooD7BIJpI'060:SH _c5NrFzp:-WW*FQiHL2YKt?uT8!kRyl[﻿La"h_uFQ.
'P9nVgPpW
RSH*b1k)5ji)gf7&(uF!,GV]5:?Wn8Ta6;t?dcQxd;UrjIeL2bNlTwZ18"SDp_cw'I
m2VBit Tdc7RJ*Ye(]XgYJazLIeJSn)5 XYqX;m17vSQ&QZ
OLjl08ywYdN'RyOG&J.[sAd?e(rG)U
f)dmxViwT4EUtT8_8iY8*cM0hq3﻿bjrAYDwDCD!U E18IFO cjz9yF
-nz)v2y3*﻿o97QGs5f8f7rM&8D81CylO4Vvri1z)5OjXnuv*7PmI&QTM3**FieV]]rAm)
;qP9.P9QigV8:v.!kwj]"((KVW*u6N1EE1
7z)5CTNq-:C0 FQ;n!x
-,iY
YYLd,m2
VQZ
JHvSqXuLSIJ3o1vD﻿(W5eJKbJqs9e*
